# Preparación de datos de test

In [1]:
import pickle as pckl

In [2]:
input_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-augmented-check/data_train.txt"
output_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-augmented-check/data_test.pckl"
mapper = {1:'Panel', 0:'Dedo'}

In [3]:
rows = []
with open(input_path) as fd:
    for item in fd:
        filename_and_boxes = item.rstrip('\n').split(' ')
        filename = filename_and_boxes[0]
        boxes = filename_and_boxes[1:]
        d = {'filename': filename, 'object':[]}
        for box in boxes:
            box = box.split(',')
            d['object'].append({'xmin':int(box[0]), 'ymin':int(box[1]), 'xmax': int(box[2]), 'ymax': int(box[3]), 'name': mapper[int(box[4])]})
        rows.append(d)

In [4]:
pckl.dump(rows, open(output_path, 'wb'))

# Dependencias

In [5]:
import argparse
import json
import pickle as pckl
import numpy as np
import os
import cv2
from PIL import Image
from scipy.special import expit
from yolo3.yolo import YOLO

## Cargar modelo

In [6]:
def load_model(model_path, classes_path, anchors_path):

    yolo = YOLO(
        **{
            "model_path": model_path,
            "anchors_path": anchors_path,
            "classes_path": classes_path,
            "score": 0.5,
            "gpu_num": 1,
            "model_image_size": (416, 416),
        }
    )

    return yolo

## Bounding boxes

In [7]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        
        self.c       = c
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score 

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3  
        
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

## Generador de lotes

In [8]:
class BatchGenerator():
    def __init__(self, instances, anchors, labels, batch_size=1, shuffle=True):
        self.instances          = instances
        self.batch_size         = batch_size
        self.labels             = labels
        self.anchors            = [BoundBox(0, 0, anchors[2*i], anchors[2*i+1]) for i in range(len(anchors)//2)]

        if shuffle:
            np.random.shuffle(self.instances)      
            
    def num_classes(self):
        return len(self.labels)

    def size(self):
        return len(self.instances)    

    def get_anchors(self):
        anchors = []

        for anchor in self.anchors:
            anchors += [anchor.xmax, anchor.ymax]

        return anchors

    def load_annotation(self, i):
        annots = []

        for obj in self.instances[i]['object']:
            annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.labels.index(obj['name'])]
            annots += [annot]

        if len(annots) == 0: annots = [[]]

        return np.array(annots)

    def load_image(self, i):
        return cv2.imread(self.instances[i]['filename'])    

## Detection

In [9]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
        
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0
                    
def get_yolo_boxes(model, images, net_h, net_w, nms_thresh):
    batch_output, data = model.detect_image(Image.fromarray(images[0].astype('uint8')))
    boxes = []
    
    for bo in batch_output:
        b = [0]*2
        b[bo[4]] = bo[5]
        box = bo[:4] + [bo[5]] + [b]
        boxes.append(BoundBox(box[0], box[1], box[2], box[3], box[4], box[5]))

    # image_h, image_w, _ = images[0].shape
    # correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    do_nms(boxes, nms_thresh)
    return [boxes]


In [10]:
def detection(model, generator, nms_thresh=0.5, net_h=416, net_w=416):  
    # gather all detections and annotations
    all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size())]
    all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        raw_image = [generator.load_image(i)]

        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, nms_thresh)[0]

        score = np.array([box.get_score() for box in pred_boxes])
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()
            
    return all_detections, all_annotations

## Evaluation

In [11]:
def compute_overlap(a, b):
    """
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    Parameters
    ----------
    a: (N, 4) ndarray of float
    b: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], 1), b[:, 1])

    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)

    ua = np.expand_dims((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih

    ua = np.maximum(ua, np.finfo(float).eps)

    intersection = iw * ih

    return intersection / ua  

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap  

In [12]:
def evaluation(all_detections, all_annotations, generator, iou_threshold=0.5):
    average_precisions = {}
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotaci�n de esa detecci�n es un falso positivo
                    false_positives = np.append(false_positives, 1) # Indicador de falso positivo
                    true_positives  = np.append(true_positives, 0) # Indicador de ausencia de falso negativo
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOI
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap         = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision
        recall    = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions[label] = {'AP': average_precision, 'recall': recall, 'precision': precision, 'f1': f1, 'support': 0}

    return average_precisions

# Evaluación

## Carga de modelo y de datos de test

In [13]:
os.chdir('/home/ubuntu/tfm')
config_path = './utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

infer_model = load_model(config['train']['model_folder'], config['train']['classes_path'], config['train']['anchors_path'])

/home/ubuntu/tfm/standalone/models/82f298920f8646ef89bd4313ef8202d9/artifacts/model/data/model.h5 model, anchors, and classes loaded in 14.88sec.


## Test

In [14]:
all_detections, all_annotations = detection(infer_model, valid_generator)

(416, 416, 3)
Found 1 boxes for img
Panel 0.68 (134, 406) (333, 466)
Time spent: 2.887sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.68 (133, 405) (334, 466)
Time spent: 0.591sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.51 (1105, 609) (1218, 667)
Panel 0.54 (1449, 603) (1601, 685)
Dedo 0.89 (713, 239) (851, 591)
Time spent: 0.564sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.53 (660, 399) (1001, 549)
Panel 0.79 (338, 472) (563, 574)
Time spent: 0.440sec
(416, 416, 3)
Found 0 boxes for img
Time spent: 0.404sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.92 (576, 362) (988, 497)
Time spent: 0.497sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.65 (179, 381) (447, 456)
Panel 0.82 (602, 371) (1057, 466)
Dedo 0.85 (1048, 483) (1335, 866)
Time spent: 0.525sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.97 (1147, 488) (1409, 956)
Time spent: 0.544sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.61 (10, 472) (406, 634)
Panel 0.66 (469, 310) (1109, 576)
Dedo 0.50 (1159, 425) (1530, 884)
Time 

In [15]:
average_precisions = evaluation(all_detections, all_annotations, valid_generator)

## Procesar salida

In [16]:
items = 0
precision = 0
for label, average_precision in average_precisions.items():
    print(labels[label] + ': {:.4f}'.format(average_precision['AP']))
    items += 1 
    precision += average_precision['AP']
print('mAP: {:.4f}'.format(precision / items))

Dedo: 0.9286
Panel: 0.4512
mAP: 0.6899
